In [ ]:
    import sqlite3
    import urllib.request, urllib.parse, urllib.error
    import xml.etree.ElementTree as ET

    fname = input('Enter file name: ')
    if ( len(fname) < 1 ) : fname = 'Library.xml'
    uh = open(fname)

    conn = sqlite3.connect('iTunesLibrary.sqlite')
    cur = conn.cursor()

    cur.executescript('''
                    CREATE TABLE IF NOT EXISTS Artist (
                                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                                name    TEXT UNIQUE);
                                
                    CREATE TABLE IF NOT EXISTS Genre (
                                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                                name    TEXT UNIQUE);
                                
                    CREATE TABLE IF NOT EXISTS Album (
                                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                                artist_id  INTEGER,
                                title   TEXT UNIQUE);
                                
                    CREATE TABLE IF NOT EXISTS Track (
                                id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
                                title TEXT  UNIQUE,
                                album_id  INTEGER,
                                genre_id  INTEGER,
                                len INTEGER, 
                                rating INTEGER, 
                                count INTEGER);
                                
                    DELETE FROM Artist;
                    DELETE FROM Genre;
                    DELETE FROM Album;
                    DELETE FROM Track
    ''') 


    def lookup(d, key):
        found = False
        for child in d:
            if found : return child.text
            if child.tag == 'key' and child.text == key :
                found = True
        return None

    stuff = ET.parse(fname)
  
    all = stuff.findall('dict/dict/dict')
    print('Dict count:', len(all))

    for entry in all:

        if ( lookup(entry, 'Track ID') is None ) : continue
        if ( lookup(entry, 'Genre') is None ) : continue

        name = lookup(entry, 'Name')
        artist = lookup(entry, 'Artist')
        album = lookup(entry, 'Album')
        count = lookup(entry, 'Play Count')
        rating = lookup(entry, 'Rating')
        length = lookup(entry, 'Total Time')
        genre = lookup(entry, 'Genre')

        if name is None or artist is None or album is None : continue

        cur.execute('''INSERT OR IGNORE INTO Artist (name) 
            VALUES ( ? )''', ( artist, ) )
        cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
        artist_id = cur.fetchone()[0]

        cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
            VALUES ( ?, ? )''', ( album, artist_id ) )
        cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
        album_id = cur.fetchone()[0]

        cur.execute('''INSERT OR IGNORE INTO Genre (name) 
            VALUES ( ? )''', (genre, ) )
        cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
        genre_id = cur.fetchone()[0]

        cur.execute('''INSERT OR REPLACE INTO Track
            (title, album_id, len, rating, count, genre_id) 
            VALUES ( ?, ?, ?, ?, ?, ? )''', 
            ( name, album_id, length, rating, count, genre_id ) )

            
    conn.commit()

    table = '''SELECT Track.title, Artist.name, Album.title, Genre.name 
                FROM Track JOIN Genre JOIN Album JOIN Artist 
                ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
                AND Album.artist_id = Artist.id
                ORDER BY Artist.name LIMIT 3'''
    for line in cur.execute(table): 
        print(str(line[0]), "|", str(line[1]),"|", str(line[2]),"|", str(line[3]))

    conn.close()